In [1]:
import numpy as np
import pandas as pd
from time import time
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score,mean_squared_error

%matplotlib inline

In [2]:
total_events=pd.read_csv('raw_data/500_users_events.csv')
action_types=['browsed','collected','carted','bought']

In [3]:
total_users=set(total_events.user_id.unique())
total_items=set(total_events.item_id.unique())
'total events %d, total users %d, total items %d' % (len(total_events),len(total_users), len(total_items))

'total events 571688, total users 500, total items 198786'

In [4]:
# behaves=total_events[['user_id','item_id','behavior_type','time']].pivot_table(index=['user_id','item_id'],columns='behavior_type',aggfunc='count').fillna(0)
# behaves.columns=action_types

In [5]:
# behaves.browsed.hist(bins=100,figsize=(20,5))

In [6]:
# 'average user events %d' % (len(total_events)//len(total_users))

In [7]:
# total_events.groupby('user_id').count()[['item_id']][lambda df:df.item_id<2000].hist(bins=100,figsize=(20,5))

In [8]:
# behaves.corr()

In [9]:
# behaves['label']=behaves.bought.apply(lambda b:1. if b>0 else 0.)

# buy_behaves=behaves[behaves.label==1]
# nbuy_behaves=behaves[behaves.label==0]

# sample_nbuy_idx=np.random.choice(len(nbuy_behaves),len(buy_behaves))
# sample_behaves=pd.concat([buy_behaves,nbuy_behaves.iloc[sample_nbuy_idx]])
# assert(len(sample_behaves[sample_behaves.label==0])==len(sample_behaves[sample_behaves.label==1]))

# X_train,X_test,y_train,y_test=train_test_split(sample_behaves[['browsed']],sample_behaves[['label']],test_size=.2)

# lr=LogisticRegression()
# lr.fit(X_train,y_train)
# roc_auc_score(y_train,lr.predict_proba(X_train)[:,1])

In [10]:
# from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler

# ss=MinMaxScaler()
# behaves['browsed_std']=ss.fit_transform(behaves[['browsed']])
# behaves['rating']=behaves['browsed_std']+behaves['label']

# behaves.describe()

In [11]:
# behaves[behaves.rating<1].rating.hist(bins=100,figsize=(20,6))

In [12]:
# bought_events=total_events[total_events.behavior_type==4]
# bought_behaves=bought_events[['user_id','item_id','behavior_type']].pivot_table(index='user_id',columns='item_id',aggfunc='count').fillna(0)
# isbought=bought_behaves.apply(lambda c:c.apply(lambda x:1 if x>0 else 0))
# isbought.columns=isbought.columns.levels[1]

In [13]:
# browsed_events=total_events[total_events.behavior_type==1]
# browsed_behaves=browsed_events[['user_id','item_id','behavior_type']].pivot_table(index='user_id',columns='item_id',aggfunc='count').fillna(0)
# browsed=pd.DataFrame(ss.fit_transform(browsed_behaves),index=browsed_behaves.index,columns=browsed_behaves.columns.levels[1])
# browsed.head()

In [14]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler

ss=MinMaxScaler()

# behave_summ=total_events[['user_id','item_id','behavior_type','time']].pivot_table(index='user_id',columns=['behavior_type','item_id'],aggfunc='count').fillna(0)

# br=behave_summ['time',1]
# bo=behave_summ['time',4]
# bo=bo.apply(lambda c:c.apply(lambda x:1 if x>0 else 0))
# br=pd.DataFrame(ss.fit_transform(br),index=br.index,columns=br.columns)

# same_cols=br.columns&bo.columns

# ratings_mat=br[same_cols]+bo[same_cols]
# ratings_mat=ratings_mat.join(bo[bo.columns^same_cols]).join(br[br.columns^same_cols])

# br.shape,bo.shape,ratings_mat.shape,np.max(ratings_mat.values.reshape(-1,1)),np.min(ratings_mat.values.reshape(-1,1))

In [15]:
br_summ=total_events[total_events.behavior_type==1].groupby(['user_id','item_id']).count()['behavior_type']
br_summ=pd.DataFrame(ss.fit_transform(br_summ.values.reshape(len(br_summ),1)),index=br_summ.index,columns=['rating'])

bo_summ=total_events[total_events.behavior_type==4].groupby(['user_id','item_id']).count()['behavior_type']
bo_summ=pd.DataFrame(bo_summ.values,index=bo_summ.index,columns=['rating']).apply(lambda c:c.apply(lambda x:1 if x>0 else 0))

same_idx=br_summ.index&bo_summ.index

ratings=br_summ.loc[same_idx]+bo_summ.loc[same_idx]
ratings=pd.concat([ratings,br_summ.loc[br_summ.index^same_idx],bo_summ.loc[bo_summ.index^same_idx]])
ratings.describe()

C:\Users\zhouqi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


,rating
count,218435.000000
mean,0.032333
std,0.157203
min,0.000000
25%,0.000000
50%,0.006211
75%,0.012422
max,2.000000


In [16]:
np.random.seed(42)
X_train,X_test,y_train,y_test=train_test_split(ratings,ratings.rating,test_size=0.1)

## base 

In [17]:
from base import BasePredictor
t=time()
bp=BasePredictor(learning_rate=0.1,alpha=0.01,max_iter=100,epsilon=1e3)
bp.fit(X_train,y_train)
print('time cost %d'%int(time()-t))
'error %.3f'%mean_squared_error(y_test,bp.predict(X_test))

{'epsilon': 100, 'alpha': 0.01, 'learning_rate': 0.1, 'max_iter': 100}
2018-02-23 19:45:41 iter 1, cost 210962.22
2018-02-23 19:47:16 iter 2, cost 161860.05
2018-02-23 19:49:05 iter 3, cost 130193.09
2018-02-23 19:50:43 iter 4, cost 107724.78
2018-02-23 19:52:22 iter 5, cost 91221.31
2018-02-23 19:54:05 iter 6, cost 78772.25
2018-02-23 19:56:01 iter 7, cost 69176.17
2018-02-23 19:57:59 iter 8, cost 61643.75
2018-02-23 19:59:54 iter 9, cost 55638.77
2018-02-23 20:01:50 iter 10, cost 50786.78
2018-02-23 20:03:47 iter 11, cost 46820.20
2018-02-23 20:05:45 iter 12, cost 43543.93
2018-02-23 20:07:48 iter 13, cost 40813.17
2018-02-23 20:10:09 iter 14, cost 38518.74
2018-02-23 20:12:20 iter 15, cost 36577.13
2018-02-23 20:14:16 iter 16, cost 34923.62
2018-02-23 20:16:10 iter 17, cost 33507.47
2018-02-23 20:18:14 iter 18, cost 32288.47
2018-02-23 20:20:17 iter 19, cost 31234.40
2018-02-23 20:22:20 iter 20, cost 30319.24
2018-02-23 20:24:24 iter 21, cost 29521.77
2018-02-23 20:26:38 iter 22, co

KeyboardInterrupt: 

In [ ]:
'error %.3f'%mean_squared_error(y_test,bp.predict(X_test))

will users appear next day?

In [20]:
total_events['time']=pd.DatetimeIndex(total_events.time)
total_events['day']=total_events.time.apply(lambda t:pd.datetime(year=t.year,month=t.month,day=t.day))
total_events['hour']=total_events.time.apply(lambda t:t.hour)

In [29]:
day1=pd.datetime(year=2014,month=12,day=17)
day2=pd.datetime(year=2014,month=12,day=18)
day1_events=total_events[total_events.day==day1]
day2_events=total_events[total_events.day==day2]

len(set(day1_events.user_id.unique())&set(day2_events.user_id.unique()))/len(day1_events.user_id.unique()),len(day2_events.user_id.unique())

(0.7896341463414634, 318)

In [34]:
day1=pd.datetime(year=2014,month=11,day=1)
day2=pd.datetime(year=2014,month=12,day=18)
day1_events=total_events[(total_events.day>=day1)&(total_events.day<day2)]
day2_events=total_events[total_events.day==day2]

len(set(day1_events.user_id.unique())&set(day2_events.user_id.unique()))/len(day1_events.user_id.unique()),len(day2_events.user_id.unique())

(0.636, 318)